# Predicting target in train

### sélection des librairies

In [1]:
import sys
import numpy as np
import pandas as pd
import sklearn as sk
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import preprocessing


#### Importation des bases de données

In [3]:
# We apply the preparation that you defined. You should not modify this.

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


Fonction d'augmentation

In [7]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

Normalisation des données 

In [ ]:
target = train['target']
features = [c for c in train.columns if c not in ['ID_code', 'target']]
normalizer = preprocessing.Normalizer().fit(train[features])
normalizer
normalizer.transform(train[features]) 

In [8]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': 1}

Modélisation

In [11]:
num_folds = 5
 

folds = KFold(n_splits=num_folds, random_state=2319)
oof = np.zeros(len(train))
getVal = np.zeros(len(train))
predictions = np.zeros(len(target))
feature_importance_df = pd.DataFrame()

In [12]:


print('Light GBM Model')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    
    X_train, y_train = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_valid, y_valid = train.iloc[val_idx][features], target.iloc[val_idx]
    
    X_tr, y_tr = augment(X_train.values, y_train.values)
    X_tr = pd.DataFrame(X_tr)
    
    print("Fold:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])
    
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    getVal[val_idx]+= clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    


Light GBM Model
Fold:1
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.913502	valid_1's auc: 0.895626
[10000]	training's auc: 0.924081	valid_1's auc: 0.89891
[15000]	training's auc: 0.932378	valid_1's auc: 0.899237
Early stopping, best iteration is:
[13778]	training's auc: 0.930457	valid_1's auc: 0.899279
Fold:2
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.914423	valid_1's auc: 0.894878
[10000]	training's auc: 0.924895	valid_1's auc: 0.898014
[15000]	training's auc: 0.933077	valid_1's auc: 0.898194
Early stopping, best iteration is:
[12177]	training's auc: 0.928571	valid_1's auc: 0.898379
Fold:3
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.913654	valid_1's auc: 0.899161
[10000]	training's auc: 0.924178	valid_1's auc: 0.901699
[15000]	training's auc: 0.932417	valid_1's auc: 0.901632
Early stopping, best iteration is:
[12089]	training's auc: 0.927785	valid_1's 

In [13]:
test = pd.read_csv("test.csv")
normalizer.transform(test[features])  
predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

CV score: 0.90083 


In [17]:
Pred2=pd.DataFrame(predictions)
Pred2['target'] = np.mean(Pred2[[col for col in Pred2.columns if col not in ['ID_code', 'target']]].values, axis=1)
sub = pd.DataFrame({"ID_code": test.ID_code.values})

sub["target"] = Pred2['target']
sub.to_csv('submission_PE.csv', index=False)

In [15]:
sub

,ID_code
0,test_0
1,test_1
2,test_2
3,test_3
4,test_4
5,test_5
6,test_6
7,test_7
8,test_8
9,test_9


That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
